<a href="https://colab.research.google.com/github/Zaka123456/MRP/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Load Dataset

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/MRP/Current/Deliverables/Methodology_&_Experiments/Report/Coding/Data_files/Data_Co.csv', encoding='latin-1')

In [ ]:
display(df.head())

,date,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,...,order_month,order_dayofweek,order_weekend,shipping_date,shipping_time,shipping_hour,shipping_week,shipping_month,shipping_dayofweek,shipping_weekend
0,2018-01-31 22:56:00,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,...,1,2,False,2018-02-03,22:56:00,22,5,2,5,True
1,2018-01-13 12:27:00,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,...,1,5,True,2018-01-18,12:27:00,12,3,1,3,False
2,2018-01-13 12:06:00,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,...,1,5,True,2018-01-17,12:06:00,12,3,1,2,False
3,2018-01-13 11:45:00,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,...,1,5,True,2018-01-16,11:45:00,11,3,1,1,False
4,2018-01-13 11:24:00,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,...,1,5,True,2018-01-15,11:24:00,11,3,1,0,False


In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])
df = df.set_index('order_date')

In [ ]:
display(df.head())

,date,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,...,order_month,order_dayofweek,order_weekend,shipping_date,shipping_time,shipping_hour,shipping_week,shipping_month,shipping_dayofweek,shipping_weekend
order_date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,2018-01-31 22:56:00,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,...,1,2,False,2018-02-03,22:56:00,22,5,2,5,True
2018-01-13,2018-01-13 12:27:00,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,...,1,5,True,2018-01-18,12:27:00,12,3,1,3,False
2018-01-13,2018-01-13 12:06:00,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,...,1,5,True,2018-01-17,12:06:00,12,3,1,2,False
2018-01-13,2018-01-13 11:45:00,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,...,1,5,True,2018-01-16,11:45:00,11,3,1,1,False
2018-01-13,2018-01-13 11:24:00,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,...,1,5,True,2018-01-15,11:24:00,11,3,1,0,False


In [ ]:
# Drop irrelevant/leaky columns
columns_to_drop = [
        'Product Description', 'Product Image', 'Order Zipcode', 'Customer Email', 'Customer Password',
        'Customer Zipcode', 'Customer Lname', 'Customer Fname', 'Days for shipping (real)',
        'Benefit per order', 'Order Item Profit Ratio', 'Order Profit Per Order', 'Sales per customer',
        'Order Item Total', 'shipping date (DateOrders)', 'shipping_hour', 'shipping_week',
        'shipping_month', 'shipping_dayofweek', 'shipping_weekend', 'Category Id',
        'Order Item Cardprod Id', 'Order Customer Id', 'Order Item Id', 'Order Item Product Price',
        'Product Card Id', 'Order Item Discount Rate', 'Product Status', 'Order City', 'Order State',
        'Customer City', 'Customer State', 'Customer Street', 'Late_delivery_risk', 'date'

]

In [ ]:
df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [ ]:
# Encode non-numeric features
import numpy as np
from sklearn.preprocessing import LabelEncoder

non_numeric_cols = df.select_dtypes(exclude=np.number).columns
processed_cols = []

for col in non_numeric_cols:
    if df[col].nunique() < 50000:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str).fillna("missing"))
        processed_cols.append(col)

print("Processed non-numeric columns:", processed_cols)

Processed non-numeric columns: ['Type', 'Delivery Status', 'Category Name', 'Customer Country', 'Customer Segment', 'Department Name', 'Market', 'Order Country', 'Order Region', 'Order Status', 'Product Name', 'Shipping Mode', 'order_time', 'order_weekend', 'shipping_date', 'shipping_time']


In [ ]:
# Resample 'Sales' to daily frequency
daily_sales = df['Sales'].resample('D').sum()

In [ ]:
# Prepare X and y
X = df.loc[:, ~df.columns.isin(["Sales"])]
y = daily_sales

In [ ]:
# Scale features
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

In [ ]:
# Resample to daily frequency
daily_scaled_X = scaled_X.resample('D').sum()

In [ ]:
 # Feature selection
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=42)
rfr.fit(daily_scaled_X, y)

RandomForestRegressor(random_state=42)

In [ ]:
importances = pd.Series(rfr.feature_importances_, index=daily_scaled_X.columns)
selected_features = importances.sort_values(ascending=False).head(20).index.tolist()

In [ ]:
print("Top selected features:", selected_features)

Top selected features: ['Order Item Discount', 'Product Price', 'Department Id', 'Latitude', 'Order Item Quantity', 'Product Name', 'Product Category Id', 'Order Region', 'Order Status', 'Category Name', 'Department Name', 'order_time', 'Longitude', 'Order Id', 'shipping_date', 'order_hour', 'Order Country', 'Customer Id', 'shipping_time', 'order_month']


In [ ]:
# Return selected daily data
df_selected = daily_scaled_X[selected_features]
df_selected["Sales"] = y

/tmp/ipython-input-21-2782659618.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["Sales"] = y


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTMCell, Dense, Dropout

# Function to create sequences
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Set time steps
TIME_STEPS = 30

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df_selected[selected_features],
    df_selected['Sales'],
    test_size=0.2,
    shuffle=False
)

# Scale the target
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# Sequence creation for LSTM
X_train_seq, y_train_seq = create_dataset(
    X_train, pd.Series(y_train_scaled.flatten(), index=y_train.index), TIME_STEPS
)
X_test_seq, y_test_seq = create_dataset(
    X_test, pd.Series(y_test_scaled.flatten(), index=y_test.index), TIME_STEPS
)

# Manual validation split
val_size = int(0.1 * len(X_train_seq))
X_val_seq = X_train_seq[-val_size:]
y_val_seq = y_train_seq[-val_size:]
X_train_seq_adj = X_train_seq[:-val_size]
y_train_seq_adj = y_train_seq[:-val_size]

# Build LSTM model using RNN(LSTMCell)
model = Sequential()
model.add(RNN(LSTMCell(50), return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dropout(0.2))
model.add(RNN(LSTMCell(50), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train
history = model.fit(
    X_train_seq_adj, y_train_seq_adj,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,
    shuffle=False
)

# Predict
y_pred_scaled = model.predict(X_test_seq)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_actual = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1))

# Metrics
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_actual, y_pred)
mape = mean_absolute_percentage_error(y_test_actual, y_pred)
r2 = r2_score(y_test_actual, y_pred)

print("\nModel Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"R-squared (R²): {r2:.4f}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 137ms/step - loss: 0.0618 - val_loss: 0.0297
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0395 - val_loss: 0.0324
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0322 - val_loss: 0.0315
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0310 - val_loss: 0.0307
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0325 - val_loss: 0.0332
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0318 - val_loss: 0.0282
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0296 - val_loss: 0.0307
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0285 - val_loss: 0.0291
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0261 - val_loss: 0.0322
Epoch 10/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0262 - val_loss: 0.0286
Epoch 11/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0229 - val_loss: 0.0308
Epoch 12/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0250 - val_loss: 0.